In [0]:
import os

In [0]:
import os

In [0]:
import pandas as pd

In [0]:
os.system("pip install -r https://raw.githubusercontent.com/George-Michael-Dagogo/World_news_tutorial/main/requirements.txt")


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


0

In [0]:
from newspaper import Article, Config

In [0]:
from nltk.corpus import stopwords

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import date, timedelta

In [0]:
from datetime import date, timedelta

In [0]:
from newsapi import NewsApiClient

In [0]:
from nltk.tokenize import word_tokenize

In [0]:
# Function to extract content from a URL
def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

In [0]:
%python
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize the News API client with an API key
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment',language='en',page_size=90,page=1)

    articles = top_headlines.get('articles', [])  # Extract articles from the API response

    # Creating DF from the articles, selecting specific columns
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy of filtered DF

    df['content'] = df['url'].apply(full_content)  # Apply the full_content function to each URL in the DataFrame
    
    df['content'] = df['content'].str.replace('\n', ' ')  # Replace newlines in the 'content' column with spaces
    
    df = df[df['content'] != 'couldnt retrieve']  # Filter out rows where the content could not be retrieved

    # Download the NLTK stopwords dataset and other required datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')

    # Apply the word count function to the 'content' column
    df['word_count'] = df['content'].apply(count_words_without_stopwords)

    
    nltk.download('vader_lexicon')

    
    sid = SentimentIntensityAnalyzer()    # Initialize the SentimentIntensityAnalyzer

    # Apply the sentiment analysis function to the 'content' column
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))

    

    return df
        
dataframe = extract_transform_function()

/home/spark-6bd0eb82-af19-4f78-a215-8e/.ipykernel/2700/command-658016692815242-3010594860:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk

In [0]:
# Function to count words excluding stopwords
def count_words_without_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

# Function for sentiment analysis
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound'], sentiment_scores['compound']

def full_content(url):
    # Dummy implementation for the full_content function
    return "This is a dummy content for the URL."

# Data extraction and transformation
def extract_transform_function():
    today = date.today()                        
    yesterday = today - timedelta(days=1)     
    day_before_yesterday = today - timedelta(days=2)   
    
    # Initialize News API client
    newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

    top_headlines = newsapi.get_top_headlines(category='entertainment', language='en', page_size=90, page=1)

    articles = top_headlines.get('articles', [])  # Extract articles

    # Convert articles to DataFrame
    init_df = pd.DataFrame(articles, columns=['source', 'title', 'publishedAt', 'author', 'url'])

    # Extract source name safely
    init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

    # Convert 'publishedAt' to date format and filter
    init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])
    filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | 
                          (init_df['publishedAt'].dt.date == yesterday)]
    
    filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
    
    df = filtered_df.copy()  # Copy DataFrame
    df['content'] = df['url'].apply(full_content)  # Fetch article content
    
    # Clean text formatting
    df['content'] = df['content'].str.replace('\n', ' ')
    
    # Remove entries where content couldn't be retrieved
    df = df[df['content'] != 'couldnt retrieve']

    # Download required NLTK datasets
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('vader_lexicon')

    # Calculate word countl
    df['word_count'] = df['content'].apply(count_words_without_stopwords)


    # Perform sentiment analysis
    df[['sentiment', 'compound_score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))


    return df

# Extract transformed data
dataframe = extract_transform_function()


/home/spark-6bd0eb82-af19-4f78-a215-8e/.ipykernel/2700/command-658016692815243-3935443897:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'publishedAt': 'date_posted'}, inplace=True)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-6bd0eb82-af19-4f78-a215-8e/nltk

In [0]:
dataframe.display()

source,title,date_posted,author,url,content,word_count,sentiment,compound_score
Hindustan Times,Kanye West poses with Bianca Censori in Spain amid shocking cousin incest confession - Hindustan Times,2025-04-22T03:34:42.000Z,Tuhin Das Mahapatra,https://www.hindustantimes.com/entertainment/music/kanye-west-poses-with-bianca-censori-in-spain-amid-shocking-cousin-incest-confession-101745289793575.html,This is a dummy content for the URL.,4,0.0,0.0
Variety,Larry David Spoofs Bill Maher's Trump Visit With Adolf Hitler Essay - Variety,2025-04-22T01:07:00.000Z,Jack Dunn,https://variety.com/2025/tv/news/larry-david-bill-maher-donald-trump-dinner-with-adolf-essay-1236374517/,This is a dummy content for the URL.,4,0.0,0.0
Cageside Seats,Becky Lynch turns heel after losing tag team titles one night after winning them - Cageside Seats,2025-04-22T00:55:31.000Z,Geno Mrosko,https://www.cagesideseats.com/wwe/2025/4/21/24413591/becky-lynch-turns-heel-raw-after-mania-lyra-valkyria-liv-morgan-raquel-rodriguez,This is a dummy content for the URL.,4,0.0,0.0
The Cut,"George and Amal Clooney Never Fight, Apparently - The Cut",2025-04-22T00:45:04.000Z,Elizabeth Gulino,http://www.thecut.com/article/george-amal-clooney-relationship-never-fight.html,This is a dummy content for the URL.,4,0.0,0.0
Cageside Seats,It looks like Randy Orton is next for WWE Champion John Cena - Cageside Seats,2025-04-22T00:24:47.000Z,Geno Mrosko,https://www.cagesideseats.com/wwe/2025/4/21/24413572/wwe-raw-after-mania-wwe-champion-john-cena-randy-orton,This is a dummy content for the URL.,4,0.0,0.0
The Washington Post,‘Andor’ and ‘Revenge of the Sith’: Two Star Wars tales about rebels and dictators - The Washington Post,2025-04-22T00:02:58.000Z,Herb Scribner,https://www.washingtonpost.com/entertainment/tv/2025/04/21/star-wars-andor-revenge-of-the-sith-resistance/,This is a dummy content for the URL.,4,0.0,0.0
Vulture,Ready or Not 2 Books a Vampire Slayer and Citizen Detective - Vulture,2025-04-21T23:43:09.000Z,Alejandra Gularte,http://www.vulture.com/article/ready-or-not-2-cast-trailer-release-date.html,This is a dummy content for the URL.,4,0.0,0.0
Wrestling Inc.,Backstage Details On TNA World Champion Joe Hendry's WWE WrestleMania 41 Appearance - Wrestling Inc.,2025-04-21T23:03:48.000Z,Ross W Berman IV,https://www.wrestlinginc.com/1840382/backstage-details-tna-world-champion-joe-hendry-wwe-wrestlemania-41-appearance/,This is a dummy content for the URL.,4,0.0,0.0
New York Post,Here’s how to watch ‘Conclave’ for free following Pope Francis’ death - New York Post,2025-04-21T20:30:00.000Z,Angela Tricarico,https://nypost.com/2025/04/21/entertainment/how-to-watch-conclave-for-free-after-pope-francis-death/,This is a dummy content for the URL.,4,0.0,0.0
Space.com,'The Fantastic Four: First Steps' trailer shown at CinemaCon now arrives for us all (video) - Space,2025-04-21T19:00:00.000Z,Jeff Spry,https://www.space.com/entertainment/the-fantastic-four-first-steps-trailer-shown-at-cinemacon-now-arrives-for-us-all-video,This is a dummy content for the URL.,4,0.0,0.0


In [0]:
print(dataframe.dtypes)

source                         object
title                          object
date_posted       datetime64[ns, UTC]
author                         object
url                            object
content                        object
word_count                      int64
sentiment                     float64
compound_score                float64
dtype: object


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS the_news;
CREATE TABLE IF NOT EXISTS the_news.news_table (
source STRING,
title STRING,
date_posted DATE,
author STRING,
url STRING,
content STRING,
word_count INT,
sentiment STRING,
compound_score DOUBLE
)

In [0]:
%sql DESCRIBE FORMATTED the_news.news_table;


col_name,data_type,comment
source,string,null
title,string,null
date_posted,date,null
author,string,null
url,string,null
content,string,null
word_count,int,null
sentiment,string,null
compound_score,double,null
,,


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType

# Initialize Spark session
spark = SparkSession.builder.appName("CreateTableExample").getOrCreate()

# Convert sentiment to string for Spark compatibility
dataframe['sentiment'] = dataframe['sentiment'].fillna('').astype(str)

# Define schema explicitly
schema = StructType([
    StructField("source", StringType(), True),
    StructField("title", StringType(), True),
    StructField("date_posted", DateType(), True), 
    StructField("author", StringType(), True),
    StructField("url", StringType(), True),
    StructField("content", StringType(), True),
    StructField("word_count", IntegerType(), True),
    StructField("sentiment", StringType(), True),
    StructField("compound_score", DoubleType(), True)
])

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(dataframe, schema=schema)

spark_df.write.mode('append').saveAsTable('the_news.news_table')


In [0]:
%sql
select * from the_news.news_table

source,title,date_posted,author,url,content,word_count,sentiment,compound_score
Hindustan Times,Kanye West poses with Bianca Censori in Spain amid shocking cousin incest confession - Hindustan Times,2025-04-22,Tuhin Das Mahapatra,https://www.hindustantimes.com/entertainment/music/kanye-west-poses-with-bianca-censori-in-spain-amid-shocking-cousin-incest-confession-101745289793575.html,This is a dummy content for the URL.,4,0.0,0.0
Deadline,Miro & CJ Barnyashev Return To WWE In New Deal As Pair Makes Surprise Appearance At Monday Night Raw - Deadline,2025-04-22,Rosy Cordero,http://deadline.com/2025/04/miro-barnyashev-wwe-deal-1236373427/,This is a dummy content for the URL.,4,0.0,0.0
Cageside Seats,Becky Lynch turns heel after losing tag team titles one night after winning them - Cageside Seats,2025-04-22,Geno Mrosko,https://www.cagesideseats.com/wwe/2025/4/21/24413591/becky-lynch-turns-heel-raw-after-mania-lyra-valkyria-liv-morgan-raquel-rodriguez,This is a dummy content for the URL.,4,0.0,0.0
The Cut,"George and Amal Clooney Never Fight, Apparently - The Cut",2025-04-22,Elizabeth Gulino,http://www.thecut.com/article/george-amal-clooney-relationship-never-fight.html,This is a dummy content for the URL.,4,0.0,0.0
The Washington Post,‘Andor’ and ‘Revenge of the Sith’: Two Star Wars tales about rebels and dictators - The Washington Post,2025-04-22,Herb Scribner,https://www.washingtonpost.com/entertainment/tv/2025/04/21/star-wars-andor-revenge-of-the-sith-resistance/,This is a dummy content for the URL.,4,0.0,0.0
TVLine,New Star Wars Series in the Works From Lost EP Carlton Cuse - TVLine,2025-04-21,Dave Nemetz,https://tvline.com/news/star-wars-tv-show-carlton-cuse-1235438425/,This is a dummy content for the URL.,4,0.0,0.0
Wrestling Inc.,Backstage Details On TNA World Champion Joe Hendry's WWE WrestleMania 41 Appearance - Wrestling Inc.,2025-04-21,Ross W Berman IV,https://www.wrestlinginc.com/1840382/backstage-details-tna-world-champion-joe-hendry-wwe-wrestlemania-41-appearance/,This is a dummy content for the URL.,4,0.0,0.0
KTLA Los Angeles,Coachella hip-hop act draws fire for anti-Israel message - KTLA,2025-04-21,Marc Sternfield,https://ktla.com/news/california/coachella-hip-hop-act-draws-fire-for-anti-israel-message/,This is a dummy content for the URL.,4,0.0,0.0
OK!,Recluse Rob Kardashian Makes Rare Appearance at Sister Kim's Easter Celebration: Photos - okmagazine.com,2025-04-21,Jaclyn Roth,https://okmagazine.com/p/recluse-rob-kardashian-rare-appearance-sister-kim-easter/,This is a dummy content for the URL.,4,0.0,0.0
New York Post,Here’s how to watch ‘Conclave’ for free following Pope Francis’ death - New York Post,2025-04-21,Angela Tricarico,https://nypost.com/2025/04/21/entertainment/how-to-watch-conclave-for-free-after-pope-francis-death/,This is a dummy content for the URL.,4,0.0,0.0
